1. Load the CSV file into a DataFrame.
2. Parse the "Radiomics" column, as it contains JSON data.
3. Remove columns with the same values across all rows.

In [12]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

# create random seed for reproducibility
random_state = 42

# Load the data from DF_Radiomics_noduls_with_diagnose.csv
file_path = "DF_Radiomics_noduls_with_diagnose.csv"
data = pd.read_csv(file_path)

# Parse the JSON in the 'Radiomics' column
data['Radiomics'] = data['Radiomics'].apply(json.loads)

# Convert the 'Radiomics' column into separate columns
radiomics_data = pd.json_normalize(data['Radiomics'])

# Drop the original 'Radiomics' column
data = data.drop('Radiomics', axis=1)

# Combine the data with the new radiomics columns
data = pd.concat([data, radiomics_data], axis=1)

# Remove columns with the same value across all rows
data = data.loc[:, (data != data.iloc[0]).any()]

# Create a stratified split
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['Labels'])

# Save the data to CSV files
train_data.to_csv('DF_Radiomics_noduls_with_diagnose_train_data.csv', index=False)
test_data.to_csv('DF_Radiomics_noduls_with_diagnose_test_data.csv', index=False)


In [15]:
print("Train data:", train_data.shape)
print("Test data:", test_data.shape)

Train data: (324, 108)
Test data: (81, 108)


# Fully Connected Neural Network

In [16]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Assuming 'data' is your pandas DataFrame
# Split data into features and labels
X = data.drop('Labels', axis=1).values
y = data['Labels'].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X).float()
y_tensor = torch.tensor(y).float()

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, stratify=y_tensor)

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)




ModuleNotFoundError: No module named 'torch'

In [ ]:
# Model
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.net(x)

# Hyperparameters
input_size = 108  # Number of features
hidden_size = 64  # You can tune this
output_size = 1   # Assuming binary classification, adjust if necessary
learning_rate = 0.001
batch_size = 32
epochs = 50  # Adjust based on your runtime requirement

# Initialize model, loss function, and optimizer
model = FCNN(input_size, hidden_size, output_size).cuda()
criterion = nn.BCEWithLogitsLoss()  # For binary classification, change if needed
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)  # L2 regularization

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)




In [ ]:
# Training loop
model.train()
for epoch in range(epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs.squeeze(), y_test_tensor)
    print(f"Test Loss: {test_loss.item()}")